Original notebook: https://www.kaggle.com/wrrosa/hubmap-tf-with-tpu-efficientunet-512x512-subm¶
Thanks to @Wojtek Rosa for sharing his notebook.
The changes I made to the notebook:

- pretrain for 4 epochs by freezing encoder layers and then train all layers at lower learning rate by using qubvel's segmentation_models set_trainable() function and encoder_freeze=True option.

- use val_loss monitoring to save best model
- the lower threshold=0.30 to determine prediction masks.
- efficientnet b5 backbone with lower batch_size = 6*8
- don't predict when committing, only on submission to save GPU time

The training notebook is [here](https://www.kaggle.com/isakev/hub-tf-tpu-valloss-pretrain-effunet5-lr1e-3-ml0-35)

OOF loss 0.022605
OOF dice_coe 0.863665
OOF accuracy 0.992854

# Versions
* V1 (V7 train notebook) 4-CV efficientunetb0 512x512 [LB .834]
* V2 (V8 train notebook) loss bce [LB .835]
* V3 (V9 train notebook) efficientunetb1 [LB .830]
* V4 (V10 train notebook) efficientunetb4 [...]

# Credits
* https://www.kaggle.com/joshi98kishan/hubmap-keras-pipeline-training-inference
* https://www.kaggle.com/leighplt/pytorch-fcn-resnet50

# Parameters
Read parameteres from notebook output, actually only **DIM** is used:

In [ ]:
mod_path = '../input/hub-tf-tpu-valloss-pretrain-effunet5-lr1e-3-ml0-35/'  # '/kaggle/input/hubmap-tf-with-tpu-efficientunet-512x512-train/'
import yaml
import pprint
with open(mod_path+'params.yaml') as file:
    P = yaml.load(file, Loader=yaml.FullLoader)
    pprint.pprint(P)

THRESHOLD = 0.3  # 0.5
WINDOW = 1024
MIN_OVERLAP = 32
NEW_SIZE = P['DIM']

# Metrics

In [ ]:
import json

with open(mod_path + 'metrics.json') as json_file:
    M = json.load(json_file)
print('Model run datetime: '+M['datetime'])
print('OOF val_dice_coe: ' + str(M['oof_dice_coe']))

# Packages

In [ ]:
! pip install ../input/kerasapplications/keras-team-keras-applications-3b180cb -f ./ --no-index -q
! pip install ../input/efficientnet/efficientnet-1.1.0/ -f ./ --no-index -q

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import gc

import numba

import rasterio
from rasterio.windows import Window

import pathlib
from tqdm.notebook import tqdm
import cv2

import tensorflow as tf
import efficientnet as efn
import efficientnet.tfkeras

In [ ]:
do_predict = False
sample_sub = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')
sample_sub.to_csv('submission.csv', index=False)

if len(sample_sub) != 5:
    do_predict = True
    
print(len(sample_sub))
sample_sub

# Functions

In [ ]:
@numba.njit()
def rle_numba(pixels):
    size = len(pixels)
    points = []
    if pixels[0] == 1: points.append(0)
    flag = True
    for i in range(1, size):
        if pixels[i] != pixels[i-1]:
            if flag:
                points.append(i+1)
                flag = False
            else:
                points.append(i+1 - points[-1])
                flag = True
    if pixels[-1] == 1: points.append(size-points[-1]+1)    
    return points

def rle_numba_encode(image):
    pixels = image.flatten(order = 'F')
    points = rle_numba(pixels)
    return ' '.join(str(x) for x in points)

def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2 
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx,ny, 4), dtype=np.int64)
    
    for i in range(nx):
        for j in range(ny):
            slices[i,j] = x1[i], x2[i], y1[j], y2[j]    
    return slices.reshape(nx*ny,4)

# Models

In [ ]:
if do_predict:
    identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
    fold_models = []
    for fold_model_path in glob.glob(mod_path+'*.h5'):
        fold_models.append(tf.keras.models.load_model(fold_model_path,compile = False))
    print(len(fold_models))

# Results

In [ ]:
if do_predict:
    p = pathlib.Path('../input/hubmap-kidney-segmentation')
    subm = {}

    for i, filename in tqdm(enumerate(p.glob('test/*.tiff')), 
                            total = len(list(p.glob('test/*.tiff')))):

        print(f'{i+1} Predicting {filename.stem}')

        dataset = rasterio.open(filename.as_posix(), transform = identity)
        slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
        preds = np.zeros(dataset.shape, dtype=np.uint8)

        for (x1,x2,y1,y2) in slices:
            image = dataset.read([1,2,3],
                        window=Window.from_slices((x1,x2),(y1,y2)))
            image = np.moveaxis(image, 0, -1)
            image = cv2.resize(image, (NEW_SIZE, NEW_SIZE))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image = np.expand_dims(image, 0)

            pred = None

            for fold_model in fold_models:
                if pred is None:
                    pred = np.squeeze(fold_model.predict(image))
                else:
                    pred += np.squeeze(fold_model.predict(image))

            pred = pred/len(fold_models)

            pred = cv2.resize(pred, (WINDOW, WINDOW))
            preds[x1:x2,y1:y2] = (pred > THRESHOLD).astype(np.uint8)

        subm[i] = {'id':filename.stem, 'predicted': rle_numba_encode(preds)}
        #print(np.sum(preds))
        del preds
        gc.collect();

# Making submission

In [ ]:
if do_predict:
    submission = pd.DataFrame.from_dict(subm, orient='index')
    submission.to_csv('submission.csv', index=False)
    submission.head()